In [1]:
import numpy as np
import plotly.graph_objects as go

from multiprocess import Pool
import os
import gzip
import pickle as pkl


def loadall_results(path, horizon, n_folds):
    result = np.zeros( (n_folds, horizon) )
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds):
            try:
                result[i] = pkl.load(f)
            except EOFError:
                break
    return result


In [ ]:
name = 'AT'
task = 'easy'
context_type = 'quintic'


n_folds = 96
horizon = 100000

colors = [  [0,0,0],  [250,0,0], [0,250,0]  ] 
labels = [  'random', 'CBPside005',  'RandCBPside005' ]
labels_latex = [  'random', 'CBPside',  'RandCBPside'  ] 

final_regrets = {}

for label, label_latex in zip(labels, labels_latex):

    direct = './contextual_results/{}'.format(name)
    path = os.path.join(direct, '{}_{}_{}_{}_{}.pkl.gz'.format(task,context_type, horizon, n_folds, label) )
    result = loadall_results(path, horizon, n_folds)
    final_regrets[label_latex] =  result[:,-1] 

final = np.array( [ (1, i) if final_regrets['RandCBPside'][i] <= final_regrets['CBPside'][i] else (0, i) for i in range(n_folds) ] )

print( sum(final[:,0]) )



In [ ]:
import numpy as np
import plotly.graph_objects as go

import games


import os
import gzip
import pickle as pkl



def loadall_results(path, horizon, n_folds):
    result = np.zeros( (n_folds, horizon) )
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds):
            try:
                result[i] = pkl.load(f)
            except EOFError:
                break
    return result

name = 'AT'
task = 'balanced'
context_type = 'linear'
epsilon = 10e-7 #0.1

n_folds = 96
horizon = 20000
d = 2


colors = [  [250,0,0],[250,0,0], [250,0,0], [250,0,0],
            [0,250,0], [0,250,0], [0,250,0], [0,250,0],
            [0,0,250], [0,0,250], [0,0,250],[0,0,250],
            [255,128,0], [255,128,0], [255,128,0], [255,128,0]  ]

labels_latex = [   'Gaussian_1_5_10e7',  'Gaussian_18_5_10e7', 'Gaussian_116_5_10e7','Gaussian_132_5_10e7', 
             'Gaussian_1_10_10e7',  'Gaussian_18_10_10e7', 'Gaussian_116_10_10e7','Gaussian_132_10_10e7',
             'Gaussian_1_20_10e7',  'Gaussian_18_20_10e7', 'Gaussian_116_20_10e7','Gaussian_132_20_10e7', 
             'Gaussian_1_100_10e7',  'Gaussian_18_100_10e7', 'Gaussian_116_100_10e7','Gaussian_132_100_10e7'  ]

labels = [ 'RandCBPside005_1_5_07', 'RandCBPside005_18_5_07', 'RandCBPside005_116_5_07', 'RandCBPside005_132_5_07' ,
                 'RandCBPside005_1_10_07', 'RandCBPside005_18_10_07', 'RandCBPside005_116_10_07', 'RandCBPside005_132_10_07',
                  'RandCBPside005_1_20_07', 'RandCBPside005_18_20_07', 'RandCBPside005_116_20_07', 'RandCBPside005_132_20_07',
                   'RandCBPside005_1_100_07', 'RandCBPside005_18_100_07', 'RandCBPside005_116_100_07', 'RandCBPside005_132_100_07'  ]

fig = go.Figure( )


for color, label, label_latex in zip( colors, labels, labels_latex):
    r,g,b = color
    direct = './contextual_results/{}'.format(name)
    path = os.path.join(direct, 'benchmark_{}_{}_{}_{}_{}.pkl.gz'.format(task,context_type, horizon, n_folds, label) )

    result = loadall_results(path, horizon, n_folds)

    worst_case = np.argmax( result[:,-1] )
    # print(worst_case)
    
    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    #std =  result[worst_case]  #np.std(result,0) 
    upper_regret = result[worst_case] #regret + std
    
    if 'Gaussian_132' in label_latex:
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'dot' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ), name='σ = 1/32'  ) ) 

    elif 'Gaussian_116' in label_latex:
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'longdashdot' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ), name='σ = 1/16'   ) )  

    elif 'Gaussian_18' in label_latex:
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'dash' ), mode='lines',
                legendgroup='M = {}'.format( label.split('_')[2] ),  name='σ = 1/8'   ) ) 

    elif 'Gaussian_1' in label_latex:
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ),legendgrouptitle_text='M = {}'.format( label.split('_')[2] ), name='σ = 1'  ) ) 

fig.update_layout(  legend= dict(yanchor="top",y=0.97,xanchor="left",x=0.03), #x = 0.25, y=0.15 for center, x= 0.97, y=0.03
                        legend_orientation="h",
                        autosize=False,
                        xaxis_title="Sequence",
                        yaxis_title="Mean Cum. Regret + Worst Cases",
                        margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ),  )
fig.show()

# fig.write_image("./benchmark_{}_{}_{}_{}.pdf".format(task, name, context_type, epsilon)  )

fig.update_yaxes(type="log")
# fig.show(legend=True)

# fig.write_image("./benchmarkRandCBPside_{}_{}_{}.pdf".format(task, name, context_type)  )
